In [1]:
import os, sys
import torch
from pathlib import Path
import numpy as np
import matplotlib
from matplotlib import cm
matplotlib.use("agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


__file__ = os.path.dirname(os.path.realpath("__file__"))
root_dir = (Path(__file__).parent / "..").resolve()
lib_dir = (root_dir / "lib").resolve()
print("The root path: {:}".format(root_dir))
print("The library path: {:}".format(lib_dir))
assert lib_dir.exists(), "{:} does not exist".format(lib_dir)
if str(lib_dir) not in sys.path:
    sys.path.insert(0, str(lib_dir))

from datasets import ConstantGenerator, SinGenerator, SyntheticDEnv
from datasets import DynamicQuadraticFunc
from datasets.synthetic_example import create_example_v1

The root path: /Users/xuanyidong/Desktop/AutoDL-Projects
The library path: /Users/xuanyidong/Desktop/AutoDL-Projects/lib


In [2]:
def draw_fig(save_dir, timestamp, xaxis, yaxis):
    save_path = save_dir / '{:04d}'.format(timestamp)
    # print('Plot the figure at timestamp-{:} into {:}'.format(timestamp, save_path))
    dpi, width, height = 40, 1500, 1500
    figsize = width / float(dpi), height / float(dpi)
    LabelSize, LegendFontsize, font_gap = 80, 80, 5

    fig = plt.figure(figsize=figsize)
    
    cur_ax = fig.add_subplot(1, 1, 1)
    cur_ax.scatter(xaxis, yaxis, color="k", s=10, alpha=0.9, label="Timestamp={:02d}".format(timestamp))
    cur_ax.set_xlabel("X", fontsize=LabelSize)
    cur_ax.set_ylabel("f(X)", rotation=0, fontsize=LabelSize)
    cur_ax.set_xlim(-6, 6)
    cur_ax.set_ylim(-40, 40)
    for tick in cur_ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        tick.label.set_rotation(10)
    for tick in cur_ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        
    plt.legend(loc=1, fontsize=LegendFontsize)
    fig.savefig(str(save_path) + '.pdf', dpi=dpi, bbox_inches="tight", format="pdf")
    fig.savefig(str(save_path) + '.png', dpi=dpi, bbox_inches="tight", format="png")
    plt.close("all")


def visualize_env(save_dir):
    save_dir.mkdir(parents=True, exist_ok=True)
    dynamic_env, function = create_example_v1(100, num_per_task=500)
    
    additional_xaxis = np.arange(-6, 6, 0.1)
    for timestamp, dataset in dynamic_env:
        num = dataset.shape[0]
        # timeaxis = (torch.zeros(num) + timestamp).numpy()
        xaxis = dataset[:,0].numpy()
        xaxis = np.concatenate((additional_xaxis, xaxis))
        # compute the ground truth
        function.set_timestamp(timestamp)
        yaxis = function(xaxis)
        draw_fig(save_dir, timestamp, xaxis, yaxis)

home_dir = Path.home()
desktop_dir = home_dir / 'Desktop'
vis_save_dir = desktop_dir / 'vis-synthetic'
visualize_env(vis_save_dir)

In [3]:
# Plot the data
cmd = 'ffmpeg -y -i {:}/%04d.png -pix_fmt yuv420p -vf fps=2 -vf scale=1000:1000 -vb 5000k {:}/vis.mp4'.format(vis_save_dir, vis_save_dir)
print(cmd)
os.system(cmd)

ffmpeg -y -i /Users/xuanyidong/Desktop/vis-synthetic/%04d.png -pix_fmt yuv420p -vf fps=2 -vf scale=1000:1000 -vb 5000k /Users/xuanyidong/Desktop/vis-synthetic/vis.mp4


0